In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [0]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.dropout=nn.Dropout(0.25)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=2, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size).cuda())

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.transitions.data[tag_to_ix["U-ADR"],tag_to_ix["B-ADR"]]=-100
        self.transitions.data[tag_to_ix["O"],tag_to_ix["B-ADR"]]=-100

        self.transitions.data[tag_to_ix["O"],tag_to_ix["I-ADR"]]=-100
        self.transitions.data[tag_to_ix["B-ADR"],tag_to_ix["I-ADR"]]=-100
        self.transitions.data[tag_to_ix["U-ADR"],tag_to_ix["I-ADR"]]=-100

        
        self.transitions.data[tag_to_ix["I-ADR"],tag_to_ix["U-ADR"]]=-100
        self.transitions.data[tag_to_ix["L-ADR"],tag_to_ix["U-ADR"]]=-100

        self.transitions.data[tag_to_ix["I-ADR"],tag_to_ix["O"]]=-100
        self.transitions.data[tag_to_ix["L-ADR"],tag_to_ix["O"]]=-100
        

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(4, 1, self.hidden_dim // 2).cuda(),
                torch.randn(4, 1, self.hidden_dim // 2).cuda())

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).cuda()
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).cuda().view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        embeds=self.dropout(embeds)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).cuda()
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).cuda(), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).cuda()
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).cuda()
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [0]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).cuda()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast).cuda()).cuda()).cuda()

In [0]:
import pickle
with open('/content/gdrive/My Drive/singular_tags.pkl', 'rb') as f:
  training_data = pickle.load(f)

with open('/content/gdrive/My Drive/test_singular_tags.pkl', 'rb') as f:
  testing_data = pickle.load(f)

In [0]:
import pickle
with open('/content/gdrive/My Drive/word_toix.pkl', 'rb') as f:
  word_to_ix = pickle.load(f)
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {"O": 0, "B-ADR": 1, "I-ADR": 2, "L-ADR": 3, "U-ADR": 4, START_TAG: 5, STOP_TAG: 6}

In [0]:
model = torch.load("/content/gdrive/My Drive/bicrf.pt")
model.eval()

BiLSTM_CRF(
  (dropout): Dropout(p=0.25, inplace=False)
  (word_embeds): Embedding(1623721, 200)
  (lstm): LSTM(200, 128, num_layers=2, bidirectional=True)
  (hidden2tag): Linear(in_features=256, out_features=7, bias=True)
)

In [0]:
!pip install sklearn_crfsuite
def acc(testing_data):
  for i in range(len(testing_data)):
      for j in range(len(testing_data[i][0])):
        word=(testing_data[i][0])[j]
        if((testing_data[i][0])[j] not in word_to_ix):
          (testing_data[i][0])[j]="<UNK>"
  # for sentence, tags in testing_data:
  #   for word in sentence:
  #       if word not in word_to_ix:
  #           word_to_ix[word] = len(word_to_ix)
  test_pred = []
  with torch.no_grad():
      for i in range(0,len(testing_data)):
        precheck_sent = prepare_sequence(testing_data[i][0], word_to_ix)
        #print(precheck_sent)
        test_pred.append(model(precheck_sent)[1])

  test_pred_adr = []
  
 #tag_to_ix = {"O": 0, "B-ADR": 1, "I-ADR": 2, "L-ADR": 3, "U-ADR": 4, START_TAG: 5, STOP_TAG: 6}
  for t in test_pred:
    temp  = []
    for k in range(len(t)):
      if t[k] == 0:
        temp.append('O')
      elif t[k] == 1:
        temp.append('B-ADR')
      elif t[k] == 2:
        temp.append('I-ADR')
      elif t[k] == 3:
        temp.append('L-ADR')
      elif t[k] == 4:
        temp.append('U-ADR')
    test_pred_adr.append(temp)
  test_adr = []
  for sent, tag in testing_data:
    test_adr.append(tag)
  import sklearn_crfsuite
  from sklearn_crfsuite import metrics
  # print("\n")
  # print("################ STRICT ####################")

  # print("f1 score macro:- ",metrics.flat_f1_score(test_adr, test_pred_adr, average='macro')) 
  # print("f1 score micro:- ",metrics.flat_f1_score(test_adr, test_pred_adr, average='micro'))

  # print("Sequence Accuracy:- ",metrics.sequence_accuracy_score(test_adr, test_pred_adr))

  # print(metrics.flat_classification_report(
  #     test_adr, test_pred_adr, labels=['I-ADR','L-ADR','U-ADR','B-ADR','O'], digits=3))
  t_adr=test_adr
  t_p_adr=test_pred_adr
  return relaxed(t_adr,t_p_adr)

     |████████████████████████████████| 747kB 3.9MB/s 


In [0]:
def relaxed(t_adr,t_p_adr):
  import sklearn_crfsuite
  from sklearn_crfsuite import metrics
  # print("\n")
  # print("################ RELAXED ####################")

  test_one=[]

  for i in range(len(t_adr)):
    temp=[]
    for j in range(len(t_adr[i])):
      if(t_adr[i][j]=='-' or t_adr[i][j]=='U-ADR'or t_adr[i][j]=='I-ADR' or t_adr[i][j]=='B-ADR' or t_adr[i][j]=='L-ADR'):
        temp.append('ADR')
      else:
        temp.append('O')
    test_one.append(temp)

  pred_one=[]


  for i in range(len(t_p_adr)):
    temp=[]
    for j in range(len(t_p_adr[i])):
      if(t_p_adr[i][j]=='-' or t_p_adr[i][j]=='U-ADR'or t_p_adr[i][j]=='I-ADR' or t_p_adr[i][j]=='B-ADR' or t_p_adr[i][j]=='L-ADR'):
        temp.append('ADR')
      else:
        temp.append('O')
    pred_one.append(temp)
  
  return pred_one, t_p_adr

  # print("f1 score macro:- ",metrics.flat_f1_score(test_one, pred_one, average='macro')) 
  # print("f1 score micro:- ",metrics.flat_f1_score(test_one, pred_one, average='micro'))

  # print("Sequence Accuracy:- ",metrics.sequence_accuracy_score(test_one, pred_one))

  # print(metrics.flat_classification_report(test_one, pred_one, labels=['ADR','O'], digits=3))

In [0]:
predictions, s_pred=acc(testing_data)

In [0]:
with open('/content/gdrive/My Drive/test_singular_tags.pkl', 'rb') as f:
  testing_data = pickle.load(f)

In [0]:
testing_data

[(['do',
   'u',
   'know',
   'what',
   'medication',
   'are',
   'r',
   'for',
   'bipolar',
   'depression',
   'currently',
   'fda',
   'approved',
   'lamotrigine',
   'aka',
   'lamictal'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']),
 (['thought',
   'of',
   'work',
   'is',
   'overwhelming',
   'me',
   'so',
   'much',
   'i',
   'feel',
   'like',
   'cry',
   'but',
   'cant',
   'because',
   'olanzapine',
   'ha',
   'me',
   'trapped',
   'in',
   'a',
   'zombie',
   'state',
   'strugglewithin'],
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'L-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'U-ADR',
   'O',
   'O']),
 (['5hrs',
   'sleep',
   'last',
   'night',
   'but',
   'stayed',
   'awake',
   'today',
   'lethargic',
   'amp',
   'low',
   'still',
   'in',
   'bed',
   'amp',
   

In [0]:
tweets=[]
for x in testing_data:
  z=x[0]
  tweets.append(z)

In [0]:
extract = []
for i in range(len(tweets)):
  temp = []
  for j in range(len(tweets[i])):
    if (s_pred[i][j] == "U-ADR" and tweets[i][j]):
      if tweets[i][j] != "<UNK>":
        temp.append([tweets[i][j]])
    else:
      if s_pred[i][j] == "B-ADR" and tweets[i][j]:
        neww = []
        k= j
        while(s_pred[i][j]!= "L-ADR"):
          if tweets[i][j]!= "<UNK>":
            neww.append(tweets[i][j])
          j+=1
        if tweets[i][j]!= "<UNK>":
            neww.append(tweets[i][j])
        newww= ' '.join(neww)
        temp.append([newww])
        
  extract.append(temp)



In [0]:
for i in range(len(extract)):
  print(i,"---", extract[i])

0 --- []
1 --- [['thought of work is overwhelming me so much i feel like cry'], ['zombie']]
2 --- [['5hrs sleep'], ['lethargic'], ['low still in bed']]
3 --- []
4 --- [['stay awake']]
5 --- [['sleep when your stomach is a cement']]
6 --- [['hungry'], ['want to eat my own arm']]
7 --- [['restart my dependence']]
8 --- [['sick'], ['tired']]
9 --- []
10 --- [['mood swing']]
11 --- []
12 --- []
13 --- []
14 --- [['hour sleep'], ['tired'], ['dizzy']]
15 --- []
16 --- [['wiping me out'], ['cant function']]
17 --- [['muscle feel stiff']]
18 --- []
19 --- [['hunger']]
20 --- [['light sleep']]
21 --- [['sleep problem']]
22 --- [['tired']]
23 --- []
24 --- [['want to throw up']]
25 --- []
26 --- [['vomit']]
27 --- [['knocked me the fuck out']]
28 --- [['drowsy'], ['dont sleep']]
29 --- []
30 --- []
31 --- []
32 --- [['sleep no problem'], ['hangover']]
33 --- [['functioning']]
34 --- []
35 --- [['interaction'], ['dizzy']]
36 --- [['sleepy']]
37 --- []
38 --- [['restless night and just want to cry

In [0]:
s_pred[297]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'U-ADR',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ADR',
 'L-ADR',
 'O',
 'O',
 'O',
 'O']

In [0]:
import nltk

extractions=[]
for i in range(len(tweets)):
  temp=[]
  for j in range(len(tweets[i])):
    if(predictions[i][j]=="ADR" and tweets[i][j]):
      temp.append(tweets[i][j])
  extractions.append(temp)



In [0]:
extractions[297]

['<UNK>', 'joint', 'pain']

In [0]:
import pickle
with open('/content/gdrive/My Drive/final_task2_onlytesting_nopred.pkl', 'rb') as f:
  testing_data = pickle.load(f)

In [0]:
testing_data['predictions'] = extract

In [0]:
testing_data

,begin,end,extraction,tweet,tokens,tags,meddra_code,meddra_term,new_intervals,predictions
tweet_id,,,,,,,,,,
326376825590779905,[nan],[nan],[nan],[do u know what medications are r for bipolar ...,"[[do, u, know, what, medication, are, r, for, ...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan],"[[nan, nan]]",[]
326406322323066883,"[111.0, 45.0]","[117.0, 71.0]","[zombie, feel like crying but can't]",[thought of work is overwhelming me so much i ...,"[[thought, of, work, is, overwhelming, me, so,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016384.0, 10011469.0]","[felt like a zombie, crying]","[[45.0, 71.0], [111.0, 117.0]]",[[thought of work is overwhelming me so much i...
326455480891346947,"[108.0, 46.0]","[122.0, 55.0]","[hot &amp; cold, Lethargic]",[5hrs sleep last night but stayed awake today....,"[[5hrs, sleep, last, night, but, stayed, awake...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016335.0, 10024262.0]","[feeling hot and cold, lethargic]","[[46.0, 55.0], [108.0, 122.0]]","[[5hrs sleep], [lethargic], [low still in bed]]"
326456822007492608,[nan],[nan],[nan],[my urologist today said that trazodone can ca...,"[[my, urologist, today, said, that, trazodone,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan],"[[nan, nan]]",[]
326575463835250689,[51.0],[56.0],[awake],[@rbhu7_82 if i don't take my quetiapine i can...,"[[if, i, dont, take, my, quetiapine, i, can, s...","[[O, O, O, O, O, O, O, O, O, U-ADR, O, O]]",[10041014.0],[sleepiness],"[[51.0, 56.0]]",[[stay awake]]
...,...,...,...,...,...,...,...,...,...,...
353707236402536449,[nan],[nan],[nan],[mark munoz plans on taking paxil inbetween ro...,"[[mark, munoz, plan, on, taking, paxil, inbetw...","[[O, O, O, O, O, O, O, O, O]]",[nan],[nan],"[[nan, nan]]",[]
353837855770030081,[nan],[nan],[nan],[@doctormcpa @sarahrnay am i reading this corr...,"[[am, i, reading, this, correctlyare, you, pre...","[[O, O, O, O, O, O, O, O, O, O, O]]",[nan],[nan],"[[nan, nan]]",[]
354228220431843328,[94.0],[100.0],[mashed],[@santypas @scrabalisios i'm taking 700mg preg...,"[[im, taking, 700mg, pregablin, and, sixty, mg...","[[O, O, O, O, O, O, O, O, O, O, O, O, U-ADR]]",[10070679.0],[feeling stoned],"[[94.0, 100.0]]",[]


In [0]:
import pickle
with open('/content/gdrive/My Drive/final_task2_predictions_joint.pkl', 'wb') as f:
  pickle.dump(testing_data, f)

In [0]:
import pickle
with open('/content/gdrive/My Drive/final_task2_predictions.pkl', 'rb') as f:
  testing_data = pickle.load(f)

In [0]:
testing_data

,begin,end,extraction,tweet,tokens,tags,meddra_code,meddra_term,new_intervals,predictions
tweet_id,,,,,,,,,,
326376825590779905,[nan],[nan],[nan],[do u know what medications are r for bipolar ...,"[[do, u, know, what, medication, are, r, for, ...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan],"[[nan, nan]]",[]
326406322323066883,"[111.0, 45.0]","[117.0, 71.0]","[zombie, feel like crying but can't]",[thought of work is overwhelming me so much i ...,"[[thought, of, work, is, overwhelming, me, so,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016384.0, 10011469.0]","[felt like a zombie, crying]","[[45.0, 71.0], [111.0, 117.0]]","[[thought, of, work, is, overwhelming, me, so,..."
326455480891346947,"[108.0, 46.0]","[122.0, 55.0]","[hot &amp; cold, Lethargic]",[5hrs sleep last night but stayed awake today....,"[[5hrs, sleep, last, night, but, stayed, awake...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[10016335.0, 10024262.0]","[feeling hot and cold, lethargic]","[[46.0, 55.0], [108.0, 122.0]]","[[5hrs, sleep], [lethargic], [low, still, in, ..."
326456822007492608,[nan],[nan],[nan],[my urologist today said that trazodone can ca...,"[[my, urologist, today, said, that, trazodone,...","[[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,...",[nan],[nan],"[[nan, nan]]",[]
326575463835250689,[51.0],[56.0],[awake],[@rbhu7_82 if i don't take my quetiapine i can...,"[[if, i, dont, take, my, quetiapine, i, can, s...","[[O, O, O, O, O, O, O, O, O, U-ADR, O, O]]",[10041014.0],[sleepiness],"[[51.0, 56.0]]","[[stay, awake]]"
...,...,...,...,...,...,...,...,...,...,...
353707236402536449,[nan],[nan],[nan],[mark munoz plans on taking paxil inbetween ro...,"[[mark, munoz, plan, on, taking, paxil, inbetw...","[[O, O, O, O, O, O, O, O, O]]",[nan],[nan],"[[nan, nan]]",[]
353837855770030081,[nan],[nan],[nan],[@doctormcpa @sarahrnay am i reading this corr...,"[[am, i, reading, this, correctlyare, you, pre...","[[O, O, O, O, O, O, O, O, O, O, O]]",[nan],[nan],"[[nan, nan]]",[]
354228220431843328,[94.0],[100.0],[mashed],[@santypas @scrabalisios i'm taking 700mg preg...,"[[im, taking, 700mg, pregablin, and, sixty, mg...","[[O, O, O, O, O, O, O, O, O, O, O, O, U-ADR]]",[10070679.0],[feeling stoned],"[[94.0, 100.0]]",[]
